In [ ]:

### FULL PECS UPDATE OF CURRENT CONTACTS ON HS

import awswrangler as wr
import numpy as np
import time
import pandas as pd

start = time.time()

con_redshift = wr.redshift.connect("redshift-cluster-matil")
con_redshift


hs_contacts = wr.redshift.read_sql_query("SELECT * FROM public.hs_contacts", con=con_redshift)

have_pecs_score = hs_contacts[~hs_contacts['pecsscore'].isnull()]

for_pecs_update = have_pecs_score[["vid","pecsscore", "email","marketing emails opened","marketing emails replied","number of times contacted","marketing emails bounced","opted out of emailmarketing information","opted out of emailone to one","unsubscribed from all email"]]
for_pecs_update.head()

for_pecs_update["new_pecs_score"] = for_pecs_update["pecsscore"]

for_pecs_update.columns = for_pecs_update.columns.str.replace('vid', 'contact_id')

step1 =  time.time()
print(step1 - start)

# Load the hs_email_activity_1 table to check how many replied to emails

hs_email_activity_1 = wr.redshift.read_sql_query("SELECT * FROM public.hs_email_activity_1", con=con_redshift)

hs_email_activity_1 = hs_email_activity_1[['contact_id','email','activity_type','activity_date','activity_assigned_to']]
hs_email_activity_1 = hs_email_activity_1[hs_email_activity_1["activity_type"] == 'Email reply from contact']
count_of_email_reply = hs_email_activity_1.groupby(['contact_id','activity_type']).count().reset_index()
count_of_email_reply.columns = count_of_email_reply.columns.str.replace('email', 'count_of_email_reply')

df = pd.merge(for_pecs_update, count_of_email_reply[["contact_id","count_of_email_reply"]], on='contact_id', how='left')

step2 =  time.time()
print(step2 - step1)



def pecs_score_update(df):
   

    global test
    
    
    # If number of times contacted > 2; then ADD +=10
    df.loc[df['number of times contacted'] > 2, 'new_pecs_score'] = df['new_pecs_score'] + 10
    
    # If marketing emails opened > 0; then ADD +=10
    df.loc[df['marketing emails opened'] > 0, 'new_pecs_score'] = df['new_pecs_score'] + 10 

    #if contact replied more than 2 time, set as 80
    
    df.loc[df['count_of_email_reply'] > 2, 'new_pecs_score'] = 80
    
    # If marketing emails replied > 1; then ADD +=10    
    df.loc[df['marketing emails replied'] > 1, 'new_pecs_score'] = df['new_pecs_score'] + 10
       
    # If opted out of emailmarketing information == True, then Set Score = 10
    df.loc[df['opted out of emailmarketing information'] == True, 'new_pecs_score'] = 10
    
    # If opted out of emailone to one == True, then Set Score = 10
    df.loc[df['opted out of emailone to one'] == True, 'new_pecs_score'] = 10
    
    # If unsubscribed from all email == True, then Set Score = 10
    df.loc[df['unsubscribed from all email'] == True, 'new_pecs_score'] = 10
    
    # If marketing emails bounced > 0, then Set Score = 10
    df.loc[df['marketing emails bounced'] > 0, 'new_pecs_score'] = 10
    
    
    #if >100 => 100
    df.loc[df['new_pecs_score'] > 100, 'new_pecs_score'] = 100
    #if <0 => 0
    df.loc[df['new_pecs_score'] < 0, 'new_pecs_score'] = 0
    
    ##Decide if to actually change the pecs score:
    
    df['change_in_hs'] = 0
        
    # If pecsscore under 60 and new_pecs_score is above 60 -> DONT change

    df.loc[(df['pecsscore'] < 60) & (df['new_pecs_score'] < 60) , 'change_in_hs'] = 0

    # If pecsscore above 60 and new_pecs_score is above 60 -> DONT change

    df.loc[(df['pecsscore'] >= 60) & (df['new_pecs_score'] >= 60) , 'change_in_hs'] = 0

    # If pecsscore under 60 and new_pecs_score is above 60 ->  change

    df.loc[(df['pecsscore'] < 60) & (df['new_pecs_score'] >= 60) , 'change_in_hs'] = 1

    # If pecsscore above 60 and new_pecs_score is under 60 ->  change

    df.loc[(df['pecsscore'] >= 60) & (df['new_pecs_score'] < 60) , 'change_in_hs'] = 1

    date_today = pd.to_datetime("today").strftime("%m%d%Y")
    
    
    test = df
    
    
    wr.s3.to_csv(
        df=df[["email","pecsscore","new_pecs_score",'change_in_hs']],
        path=f's3://panther-data-lake/lakeshore/hubspot/contacts/after-sagemaker-features/pecs-score-feature/pecs-score-before-databrew/updating_pecs_score/pecs_update_{date_today}.csv',)

pecs_score_update(df)

end = time.time()
print(end - start)